In [ ]:
import paramiko 

import pandas as pd

import data

In [ ]:
df_runup = pd.read_excel('runup_pg_dev.xlsx')

In [ ]:
len(df_runup)

In [ ]:
df_runup.head(5)

In [ ]:
stand_type = df_runup['STAND_TYPE']
ir_name_short = df_runup['IR_NAME_SHORT']
db_name = df_runup['DB_NAME']
dns = df_runup['FQDN']
ip = df_runup['IP_A']
user = 'efs_dev'  # oracle, efs_test (dev), efs_dev
path_key = 'C:/Users/19397737/Desktop/БД/dbpdevopsteam_sigma'
port = 22

bash_string = 'lsblk -no NAME,TYPE,SIZE'   # lsblk -no NAME,SIZE,MOUNTPOINT

df_bash = pd.DataFrame(columns=['STAND_TYPE', 'IR_NAME_SHORT', 'URL', 'db_name', 'User', 'Text', 'Error_py'])

In [ ]:
%%time 
for i in range(len(ip)):
    try: 
        e = None
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname=ip[i], username=user, key_filename=path_key, port=port)
        stdin, stdout, stderr = client.exec_command(bash_string)
        data = stdout.read() + stderr.read()
        client.close()
        
        df_bash = df_bash.append({'STAND_TYPE':stand_type[i], 'IR_NAME_SHORT':ir_name_short[i], 'URL':dns[i],
                                    'db_name':db_name[i], 'User':user, 
                                    'Text':data.decode("utf-8"),'Error_py': e}, ignore_index=True)
        print(i)
        
    except Exception as e:
        df_bash = df_bash.append({'STAND_TYPE':stand_type[i], 'IR_NAME_SHORT':ir_name_short[i], 'URL':dns[i],
                                    'db_name':db_name[i], 'User':user, 
                                    'Text': 'Nan', 'Error_py': e}, ignore_index=True)
        print('text_error:', e)
        
        
df_bash.to_excel('lsblk_pg_dev.xlsx')

In [ ]:
df_xlsx_lsblk = pd.read_excel('lsblk_pg_dev.xlsx')

In [ ]:
df_xlsx_lsblk.head(5)

In [ ]:
stand_type = df_xlsx_lsblk['STAND_TYPE']
ir_name_short = df_xlsx_lsblk['IR_NAME_SHORT']
url = df_xlsx_lsblk['URL']
db_name = df_xlsx_lsblk['db_name']
user = df_xlsx_lsblk['User']
text = df_xlsx_lsblk['Text']


df_output_filter = pd.DataFrame(columns=['STAND_TYPE', 'IR_NAME_SHORT','URL', 'db_name', 'User', 'name_empty_disk', 'size_empty_disk'])

In [ ]:
for i in range(len(text)):
    split_value_text = text[i].split('\n')
    for i1 in range(len(split_value_text)):
        if 'disk' in split_value_text[i1]:
            if ('├' in split_value_text[i1+1]) or ('└' in split_value_text[i1+1]):
                continue
            else: 
                print(str(i) + ' ' + str(split_value_text[i1]) + ' ' + str(url[i]))
                split_value_probel = split_value_text[i1].split(' ')
                filter_value = list(filter(lambda x: x != '', split_value_probel))
                df_output_filter = df_output_filter.append({'STAND_TYPE':stand_type[i], 'IR_NAME_SHORT':ir_name_short[i],
                                                            'URL':url[i],'db_name':db_name[i],'User':user[i],
                                                            'name_empty_disk':filter_value[0],'size_empty_disk':filter_value[2]}
                                                            ,ignore_index=True)               
df_output_filter.to_excel('pg_ift.xlsx')                

In [ ]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=ip, username=user, key_filename=path_key, port=port)
stdin, stdout, stderr = client.exec_command('ssh -4 localhost')
data = stdout.read() + stderr.read()
client.close()

In [ ]:
data.decode("utf-8")

In [ ]:
ssh = paramiko.SSHClient()

ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('10.55.72.41', username='oracle', key_filename=path_key)

stdin, stdout, stderr = ssh.exec_command('ls -i')
#print (stdin.readlines())
print (stdout.readlines())
print (stderr.readlines())
ssh.close()